In [ ]:
!pip install -U hopsworks --quiet

In [34]:
import numpy as np
import pandas as pd

#import transactions_fraud

#import great_expectations
# Mute warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import kagglehub
import shutil

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sentence_transformers import SentenceTransformer

Download csv file from kaggle, then move it

In [ ]:
path = kagglehub.dataset_download("gpreda/bbc-news")
print("Path to dataset files:", path) #"/home/jupyter/.cache/kagglehub/datasets/gpreda/bbc-news/versions/100" 
shutil.move(path, './')

Extract the data

In [15]:
def get_backfill():
    file_path = './bbc_news.csv'
    df = pd.read_csv(file_path)
    df = df[['pubDate', 'title']]
    df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S GMT')
    # as 'YYYY-MM-DD'
    df['pubDate'] =  pd.to_datetime(df['pubDate'].dt.strftime('%Y-%m-%d'))
    df2 = df[df['pubDate'] > pd.to_datetime('2023-12-29')]
    print(df2.head())
    print(df.tail())
    print(len(df))
    return df2
df_headlines = get_backfill()

         pubDate                                              title
27191 2023-12-30   Man charged with murder over Sheffield car crash
27242 2023-12-30  Ukraine war: Kremlin says 20 dead after attack...
27243 2023-12-31  Inside Russia as war in Ukraine grinds into ne...
27244 2023-12-30     Tom Wilkinson: The Full Monty actor dies at 75
27245 2023-12-30  Eurostar services to resume after major disrup...
         pubDate                                              title
41961 2024-11-29  Ditcheva first British woman to win MMA world ...
41962 2024-11-29  UK's first Dignitas widow feels ‘wonderful’ af...
41963 2024-11-29  Tight three-way race in Irish general election...
41964 2024-11-29  Could police have handled Skye shooting differ...
41965 2024-11-29        Angry doctors owed thousands refuse to work
41966


Get the embedding model

In [16]:
emb_model = SentenceTransformer('all-MiniLM-L6-v2')

2024-12-30 09:27:33,591 INFO: Use pytorch device_name: cpu
2024-12-30 09:27:33,592 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [17]:
KEY = "wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB"
project = hopsworks.login(api_key_value=KEY)

2024-12-30 09:30:53,682 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-30 09:30:53,692 INFO: Initializing external client
2024-12-30 09:30:53,694 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-30 09:30:54,838 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163414


In [23]:
embeddings = emb_model.encode(df_headlines['title'].tolist())

Batches:   0%|          | 0/460 [00:00<?, ?it/s]

In [24]:
print(embeddings.shape)

(14691, 384)


Write the embeddings, headlines and dates to hopsworks

In [ ]:
df = pd.DataFrame(embeddings, columns=[f"dim_{i}" for i in range(384)])
print(df)
df["headlines"] = df_headlines['title'].tolist()
df["date"] = df_headlines['pubDate'].tolist()
feature_group = fs.get_or_create_feature_group(
    name="headlinesemb",
    version=1,
    description="Feature group for 384-dimensional vectors",
    primary_key=["headlines"], 
    online_enabled=True 
)
feature_group.insert(df, write_options={"wait_for_job": True})
print("384-dimensional vectors successfully written to Hopsworks Feature Store.")